In [58]:
# Get party information and Twitter UIDs for politicians
# Both CSVs were downloaded from http://twitterpoliticians.org/download

import pandas as pd
import tweepy as tw
import numpy as np
from os import environ as env

In [15]:
# # Import data
# members_raw = pd.read_csv(
#     "data/full_member_info.csv", encoding="utf16", engine="python"
# )

# # Keep relevant columns
# members = members_raw[
#     [
#         "country",
#         "name",
#         "party",
#         "uid",
#         "legislative_start_date",
#         "legislative_end_date",
#     ]
# ].copy()

# # Keep only American politicians
# members = members[members["country"]=="United States"]

# # Fill NAs with 0 so we can convert from float to int
# members = members.fillna(0)

# # Convert to int
# members["uid"] = members["uid"].astype(int)

# # Start with only getting tweets for notable politicians


In [3]:
# Load credentials from .env file
auth = tw.AppAuthHandler(env["CONSUMER_KEY"], env["CONSUMER_SECRET"])

# Authorize
api = tw.API(auth)

In [61]:
SCREEN_NAMES = [
    "SpeakerPelosi",
    "RepMaxineWaters", 
    "SenTedCruz", 
    "RandPaul",
    "GOPLeader", 
    "Jim_Jordan", 
    "RepAdamSchiff", 
    "ewarren"  
]

DEMOCRATS = [
    "SpeakerPelosi",
    "RepMaxineWaters",
    "RepAdamSchiff", 
    "ewarren"
]

MAX_TWEETS = 10

dfs_list = []
for name in SCREEN_NAMES:
    try:
        # Creation of query method using parameters
        tweets = tw.Cursor(api.user_timeline, screen_name=name, tweet_mode="extended").items(MAX_TWEETS)

        # Post-process tweets
        tweets_json = [t._json for t in tweets]
        tweets_normalized = pd.json_normalize(tweets_json)
        dfs_list.append(tweets_normalized)
    except:
        print('Unable to find tweets for {}'.format(name))
      
# Combine all dfs into one df and clean
tweets_df = pd.concat(dfs_list)
tweets_df = tweets_df[["created_at", "id", "full_text", "user.id_str", "user.name", "user.screen_name"]]

# tweets_df.rename({"id": "tweet_id",
#     "user.id_str": "uid",
#     "user.name": "username", 
#     "user.screen_name": "screen_name",
#     "user.url": "user_url"
# }, inplace=True)

# Add column for political party
tweets_df['party'] = np.where(tweets_df['user.screen_name'].isin(DEMOCRATS) , 'democrat', 'republican')

,created_at,id,full_text,user.id_str,user.name,user.screen_name
0,Wed Oct 27 15:01:53 +0000 2021,1453376411104956418,"Each day, let us pledge to build a future wher...",15764644,Nancy Pelosi,SpeakerPelosi
1,Wed Oct 27 15:01:53 +0000 2021,1453376409833979908,"Over the last several years, Jewish communitie...",15764644,Nancy Pelosi,SpeakerPelosi
2,Wed Oct 27 15:01:52 +0000 2021,1453376408542191616,"Today, Americans join the Jewish community of ...",15764644,Nancy Pelosi,SpeakerPelosi
3,Tue Oct 26 20:00:49 +0000 2021,1453089251999031298,"For three decades, he has offered moral and sp...",15764644,Nancy Pelosi,SpeakerPelosi
4,Tue Oct 26 20:00:46 +0000 2021,1453089240640860162,"As House Speaker, it was an honor to welcome H...",15764644,Nancy Pelosi,SpeakerPelosi


In [4]:
# # Run a test query
# USER_ID = "17976923"
# MAX_TWEETS = 150

# # Creation of query method using parameters
# tweets = tw.Cursor(api.user_timeline, user_id=USER_ID, tweet_mode="extended").items(MAX_TWEETS)

# # Post-process tweets
# json_tweets = [t._json for t in tweets]
# tweets_df = pd.json_normalize(json_tweets)